In [1]:
import vtk
vtk.vtkOutputWindow.SetGlobalWarningDisplay(False)

In [3]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import scipy.io
import re

from brainspace.datasets import load_group_fc, load_parcellation, load_conte69
from brainspace.gradient import GradientMaps
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels

from scipy import io
import os

In [7]:
import os

import nibabel as nib
from nilearn.datasets import fetch_surf_fsaverage

import brainstat
from brainstat.stats.terms import FixedEffect
import pandas as pd

import matplotlib
matplotlib.use('Agg')

import scipy as sp
import scipy.stats
from scipy import stats

from math import pi
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

In [8]:
from brainspace.datasets import load_group_fc, load_parcellation, load_conte69

# First load mean connectivity matrix and Schaefer parcellation
conn_matrix = load_group_fc('schaefer', scale=300)
labeling = load_parcellation('schaefer', scale=300, join=True)

mask = labeling != 0

# and load the conte69 surfaces
surf_lh, surf_rh = load_conte69()

In [16]:
def expand_matrices(matrix):
    """
    Expand a single 2D matrix by inserting rows and columns of zeros at specified positions.
    
    Args:
        matrix (numpy.ndarray): 2D matrix to expand (shape N x N).
    
    Returns:
        numpy.ndarray: Expanded 2D matrix.
    """
    if matrix.ndim != 2:
        raise ValueError(f"Input matrix must be 2D, but got shape {matrix.shape}")
    
    insert_positions = [0, 4, 36, 40]
    insert_positions = sorted(insert_positions, reverse=True)
    
    expanded_matrix = matrix
    for pos in insert_positions:
        expanded_matrix = np.insert(expanded_matrix, pos, 0, axis=0)  # Insert row
        expanded_matrix = np.insert(expanded_matrix, pos, 0, axis=1)  # Insert column
    
    return expanded_matrix


In [ ]:
from tqdm import tqdm
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

fold_results = {}

for fold in range(5):
    top_folder_path = '/Final_project/241207_v1'
    fold_path = os.path.join(top_folder_path, f'Fold_{fold+1}')
    
    whole_subjects = [f for f in os.listdir(fold_path) if f.endswith('.mat') and f.startswith('sample_')]
    
    gradient_corr = {1: [], 2: [], 3: []}
    gradient_mse = {1: [], 2: [], 3: []}
    
    for subject in tqdm(whole_subjects, desc=f"Processing Fold {fold+1}"):
        subject_path = os.path.join(fold_path, subject)
        mat_data = scipy.io.loadmat(subject_path)
        
        real_FC = mat_data['SC']
        real_FC = expand_matrices(real_FC)
        gen_FC = mat_data['FC']
        gen_FC = expand_matrices(gen_FC)
        
        gm_real = GradientMaps(kernel='normalized_angle', approach='dm', random_state=0)
        gm_real.fit(real_FC)

        mask = labeling != 0

        grad_real = [None] * 3
        for i in range(3):
            grad_real[i] = gm_real.gradients_[:, i]
        
        gm_gen = GradientMaps(kernel='normalized_angle', approach='dm', random_state=0)
        gm_gen.fit(gen_FC)
        
        grad_gen = [None] * 3
    
        for i in range(3):
            grad_gen[i] = gm_gen.gradients_[:, i]

        for i in range(3):
            grad_real_flat = grad_real[i][~np.isnan(grad_real[i])]
            grad_gen_flat = grad_gen[i][~np.isnan(grad_gen[i])]

            corr, _ = pearsonr(grad_real_flat, grad_gen_flat)
            gradient_corr[i + 1].append(corr)

            mse = mean_squared_error(grad_real_flat, grad_gen_flat)
            gradient_mse[i + 1].append(mse)

        for i in range(3):
            grad_real_flat = grad_real[i][~np.isnan(grad_real[i])]
            grad_gen_flat = grad_gen[i][~np.isnan(grad_gen[i])]

            corr, _ = pearsonr(grad_real_flat, grad_gen_flat)
            gradient_corr[i + 1].append(corr)

            mse = mean_squared_error(grad_real_flat, grad_gen_flat)
            gradient_mse[i + 1].append(mse)

    fold_results[f"Fold_{fold+1}"] = {}

    for i in range(3):
        fold_results[f"Fold_{fold+1}"][f"grad_{i+1}_mean_mse"] = np.mean(gradient_mse[i+1])
        fold_results[f"Fold_{fold+1}"][f"grad_{i+1}_mean_corr"] = np.mean(gradient_corr[i+1])

results_save_path = os.path.join(top_folder_path, "gradient_results.txt")
with open(results_save_path, "w") as file:
    for fold, metrics in fold_results.items():
        print(f"{fold} Results:")
        file.write(f"{fold} Results:\n")
        for gradient, value in metrics.items():
            print(f"  {gradient}: {value:.4f}")
            file.write(f"  {gradient}: {value:.4f}\n")

Processing Fold 5: 100%|██████████| 197/197 [00:01<00:00, 182.86it/s]

Fold_1 Results:
  grad_1_mean_mse: 0.0156
  grad_1_mean_corr: 0.0197
  grad_2_mean_mse: 0.0102
  grad_2_mean_corr: 0.0761
  grad_3_mean_mse: 0.0062
  grad_3_mean_corr: -0.0099
Fold_2 Results:
  grad_1_mean_mse: 0.0152
  grad_1_mean_corr: 0.0064
  grad_2_mean_mse: 0.0092
  grad_2_mean_corr: 0.0780
  grad_3_mean_mse: 0.0057
  grad_3_mean_corr: -0.0119
Fold_3 Results:
  grad_1_mean_mse: 0.0147
  grad_1_mean_corr: 0.0124
  grad_2_mean_mse: 0.0087
  grad_2_mean_corr: 0.1072
  grad_3_mean_mse: 0.0056
  grad_3_mean_corr: 0.0029
Fold_4 Results:
  grad_1_mean_mse: 0.0144
  grad_1_mean_corr: 0.0128
  grad_2_mean_mse: 0.0093
  grad_2_mean_corr: 0.1262
  grad_3_mean_mse: 0.0057
  grad_3_mean_corr: -0.0013
Fold_5 Results:
  grad_1_mean_mse: 0.0155
  grad_1_mean_corr: 0.0131
  grad_2_mean_mse: 0.0094
  grad_2_mean_corr: 0.0787
  grad_3_mean_mse: 0.0059
  grad_3_mean_corr: 0.0115
